In [32]:
import os
os.environ['TF_NUM_INTRAOP_THREADS'] = '1'
os.environ['TF_NUM_INTEROP_THREADS'] = '1'

In [33]:

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [34]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

## 문제 해결할 것 : 너무 느림 
WARNING:absl:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, 
please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.
WARNING:absl:There is a known slowdown when using v2.11+ Keras optimizers on M1/M2 Macs. Falling back to the legacy Keras optimizer, i.e., 
`tf.keras.optimizers.legacy.Adam`.

In [46]:
tf.keras.optimizers.Adam = tf.keras.optimizers.legacy.Adam

In [47]:
import autokeras as ak

In [48]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

(60000, 28, 28)
(60000,)
[5 0 4]


In [49]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 1 Complete [00h 03m 46s]
val_loss: 0.03944199159741402

Best val_loss So Far: 0.03944199159741402
Total elapsed time: 00h 03m 46s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.1649 - accuracy: 0.9497
Epoch 2/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0746 - accuracy: 0.9765
Epoch 3/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0607 - accuracy: 0.9815
Epoch 4/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0513 - accuracy: 0.9836
Epoch 5/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0455 - accuracy: 0.9854
Epoch 6/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0395 - accuracy: 0.9876
Epoch 7/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0378 - accuracy: 0.9875
Epoch 8/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0338 - accuracy: 0.9890
Epoch 9/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0322 - accuracy: 0.9895
Epoch 10/10
1875/1875 [==============================] - 26s 14m

INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


313/313 [==============================] - 1s 3ms/step
[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]
313/313 [==============================] - 1s 3ms/step - loss: 0.0346 - accuracy: 0.9897
[0.03460265323519707, 0.9897000193595886]


In [50]:
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)

In [51]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=10,
)

In [52]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)

output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1,
)

## clf.fit(x_train, y_train, epochs=2)

In [53]:
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 53m 35s]
val_loss: 0.08372114598751068

Best val_loss So Far: 0.08372114598751068
Total elapsed time: 00h 53m 35s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
1563/1563 [==============================] - 387s 247ms/step - loss: 0.3346 - accuracy: 0.9183
Epoch 2/10
1563/1563 [==============================] - 387s 248ms/step - loss: 0.1903 - accuracy: 0.9518
Epoch 3/10
1563/1563 [==============================] - 386s 247ms/step - loss: 0.2012 - accuracy: 0.9534
Epoch 4/10
1563/1563 [==============================] - 385s 246ms/step - loss: 0.1554 - accuracy: 0.9618
Epoch 5/10
1563/1563 [==============================] - 385s 247ms/step - loss: 0.1592 - accuracy: 0.9626
Epoch 6/10
1563/1563 [==============================] - 384s 245ms/step - loss: 0.0994 - accuracy: 0.9743
Epoch 7/10
1563/1563 [==============================] - 384s 246ms/step - loss: 0.1434 - accuracy: 0.9647
Epoch 8/10
1563/1563 [==============================] - 385s 246ms/step - loss: 0.0999 - accuracy: 0.9744
Epoch 9/10
1563/1563 [==============================] - 385s 246ms/step - loss: 0.0508 - accuracy: 0.9845
Epoch 10/10
1563/1563 [=======================

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [54]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 52m 26s]
val_loss: 0.08987890183925629

Best val_loss So Far: 0.08987890183925629
Total elapsed time: 00h 52m 26s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
1563/1563 [==============================] - 380s 242ms/step - loss: 0.3250 - accuracy: 0.9066
Epoch 2/10
1563/1563 [==============================] - 380s 243ms/step - loss: 0.1894 - accuracy: 0.9506
Epoch 3/10
1563/1563 [==============================] - 386s 247ms/step - loss: 0.1374 - accuracy: 0.9621
Epoch 4/10
1563/1563 [==============================] - 385s 246ms/step - loss: 0.1321 - accuracy: 0.9643
Epoch 5/10
1563/1563 [==============================] - 387s 247ms/step - loss: 0.1338 - accuracy: 0.9657
Epoch 6/10
1563/1563 [==============================] - 390s 249ms/step - loss: 0.0953 - accuracy: 0.9736
Epoch 7/10
1563/1563 [==============================] - 380s 243ms/step - loss: 0.0886 - accuracy: 0.9746
Epoch 8/10
1563/1563 [==============================] - 384s 246ms/step - loss: 0.0724 - accuracy: 0.9794
Epoch 9/10
1563/1563 [==============================] - 391s 250ms/step - loss: 0.0700 - accuracy: 0.9800
Epoch 10/10
1563/1563 [=======================

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [55]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have the channel dimension.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

# One-hot encode the labels.
eye = np.eye(10)
y_train = eye[y_train]
y_test = eye[y_test]

print(x_train.shape)  # (60000, 28, 28, 1)
print(y_train.shape)  # (60000, 10)
print(y_train[:3])
# array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
#        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

(60000, 28, 28, 1)
(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [56]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,)))
test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,)))

clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set, epochs=10)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))

Trial 1 Complete [00h 04m 08s]
val_loss: 0.04134426638484001

Best val_loss So Far: 0.04134426638484001
Total elapsed time: 00h 04m 08s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
1875/1875 [==============================] - 29s 16ms/step - loss: 0.1565 - accuracy: 0.9524
Epoch 2/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0747 - accuracy: 0.9764
Epoch 3/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0567 - accuracy: 0.9827
Epoch 4/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0499 - accuracy: 0.9843
Epoch 5/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0454 - accuracy: 0.9857
Epoch 6/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0400 - accuracy: 0.9874
Epoch 7/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0377 - accuracy: 0.9881
Epoch 8/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0327 - accuracy: 0.9893
Epoch 9/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0309 - accuracy: 0.9900
Epoch 10/10
1875/1875 [==============================] - 27s 15m

INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


313/313 [==============================] - 1s 3ms/step - loss: 0.0361 - accuracy: 0.9880
[0.036133479326963425, 0.9879999756813049]
